<a href="https://colab.research.google.com/github/Yukselendincer/datasceinceproject/blob/main/Lojistik_Operasyonlar%C4%B1nda_Yapay_Zeka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openrouteservice

In [ ]:
import openrouteservice
import pandas as pd
import time
import os

!pip install openrouteservice

# --- AYARLAR ---
ORS_API_KEY = "eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6IjhiZDc3ZjM5MzU3NjQ3NTlhMmFiZjRmMTM2ZDBlOWVjIiwiaCI6Im11cm11cjY0In0="
client = openrouteservice.Client(key=ORS_API_KEY)

# Operasyonel Parametreler
MAZOT_FIYATI = 53.15
YAKIT_TUKETIMI_100KM = 35
KAR_MARJI = 1.25  # %25 Kar
KDV_ORANI = 1.20   # %20 KDV

rotalar = [
    {"isim": "Ambarli Limani - Corlu", "kalkis": [40.9648, 28.6745], "varis": [41.1560, 27.8180]},
    {"isim": "Gebze OSB - Cerkezkoy", "kalkis": [40.8502, 29.4280], "varis": [41.2850, 28.0010]}
]

sonuclar = []

def hesapla_lojistik_maliyet():
    print("🚀 Lojistik maliyet hesaplaması başlatıldı...\n")
    for rota in rotalar:
        try:
            # Koordinatları ORS için [Boylam, Enlem] sırasına çeviriyoruz
            coords = [(rota['kalkis'][1], rota['kalkis'][0]), (rota['varis'][1], rota['varis'][0])]

            # Ağır Vasıta (HGV) profilini kullanarak rota hesaplama
            route = client.directions(coordinates=coords, profile='driving-hgv', format='geojson')

            summary = route['features'][0]['properties']['summary']
            mesafe_km = summary['distance'] / 1000
            sure_saat = summary['duration'] / 3600

            # Maliyet ve Teklif Hesaplamaları
            yakit_maliyeti = (mesafe_km / 100) * YAKIT_TUKETIMI_100KM * MAZOT_FIYATI
            teklif_bedeli = (yakit_maliyeti * KAR_MARJI) * KDV_ORANI

            sonuclar.append({
                "Rota": rota['isim'],
                "Mesafe_KM": round(mesafe_km, 2),
                "Sure_Saat": round(sure_saat, 2),
                "Yakit_Maliyeti_TL": round(yakit_maliyeti, 2),
                "Teklif_Bedeli_KDV_Dahil": round(teklif_bedeli, 2),
                "Google_Maps_Navigasyon": f"https://www.google.com/maps/dir/{rota['kalkis'][0]},{rota['kalkis'][1]}/{rota['varis'][0]},{rota['varis'][1]}/"
            })
            print(f"✅ Başarılı: {rota['isim']} ({round(mesafe_km, 2)} KM)")
            time.sleep(1) # API limit koruması

        except Exception as e:
            print(f"❌ Hata ({rota['isim']}): {e}")

hesapla_lojistik_maliyet()

if sonuclar:
    df = pd.DataFrame(sonuclar)
    df.to_csv("lojistik_maliyet_tablosu.csv", index=False, encoding='utf-8-sig')
    print(f"\n📂 {len(sonuclar)} rota için veri kaydedildi. Dosya: lojistik_maliyet_tablosu.csv")